### 最小白的做法，只能匹配一个 pattern，并且一旦 pattern 符合之后，后续的部分不再考虑

In [1]:
def is_variable(pat):
    return pat.startswith('?') and all(p.isalpha() for p in pat[1:])

In [2]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return True
    else:
        if pattern[0] != saying[0]:
            return False
        else:
            return pat_match(pattern[1:], saying[1:])

In [3]:
pat_match('I want ?C'.split(), 'I want candy'.split())

True

In [4]:
is_variable('?I')

True

### 将符合的 pattern 打印出来

In [5]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        print(pattern[0], saying[0])
    else:
        if pattern[0] != saying[0]:
            return False
        else:
            return pat_match(pattern[1:], saying[1:])

In [6]:
pat_match('?I want ?C'.split(), 'I want candy'.split())

?I I


### 由于上述代码只能打印出一个 pattern，不适用于多个 pattern 场景，所以改进代码

In [7]:
def pat_match(pattern, saying):
    if not pattern or not saying:
        return []
    else:
        if is_variable(pattern[0]):
            return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
        else:
            if pattern[0] != saying[0]:
                return []
            else:
                return pat_match(pattern[1:], saying[1:])

In [8]:
pat_match('?I want ?C haha'.split(), 'I want candy do you'.split())

[('?I', 'I'), ('?C', 'candy')]

### 下面依次定义两个函数，一个是将 pattern 对映射进字典里面，另一个是完成 pattern 替换

In [9]:
def pat_to_dic(pattern):
    return {k: v for k, v in pattern}

dic.get(a, a) 这句话的意思是，如果字典中包含 a，直接取出 value 值，否则直接返回 a 值

In [10]:
def substitude(rule, parsed_rules):
    if not rule:
        return []
    else:
        return [parsed_rules.get(rule[0], rule[0])] + substitude(rule[1:], parsed_rules)

In [11]:
got_patterns = pat_match('I want ?I'.split(), 'I want iPhone'.split())

In [12]:
got_patterns

[('?I', 'iPhone')]

In [13]:
parsed_rules = pat_to_dic(got_patterns)

In [14]:
parsed_rules

{'?I': 'iPhone'}

In [15]:
substitude('Do you want an ?I'.split(), parsed_rules)

['Do', 'you', 'want', 'an', 'iPhone']

### 下面将上面的输出从 list 变成一句话

In [16]:
def substitude(rule, parsed_rules):
    if not rule:
        return None
    else:
        return ' '.join(parsed_rules.get(p, p) for p in rule)

In [17]:
def substitude_eng(rule, parsed_rules):
    if not rule:
        return None
    else:
        return ' '.join(parsed_rules.get(p, p) for p in rule)

In [18]:
def substitude_chi(rule, parsed_rules):
    if not rule:
        return None
    else:
        string = ''.join(parsed_rules.get(p, p) for p in rule)
        return string.replace(' ', '')

In [19]:
substitude('Do you want an ?I'.split(), parsed_rules)

'Do you want an iPhone'

### Segment match 用以匹配多个单词组合而成的 pattern

In [20]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(p.isalpha() for p in pattern[2:])

In [21]:
from collections import defaultdict

In [22]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [23]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)

In [24]:
def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [25]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [26]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

# 下面是自己实现的代码

## 问题1： 只需要考虑英文场景，需要对 pat_match_with_seg 函数进行更改

In [27]:
import random

In [28]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not saying:
        if pattern == []:
            return []
        else:
            return fail
    
    pat = pattern[0]
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [29]:
def get_response(saying, rules=rule_responses):
    for k in rules:
        match_result = pat_match_with_seg(k.split(), saying.split())
        if None not in match_result:
            response_pattern = random.choice(rules[k])
            match_result = pat_to_dict(match_result)
            return substitude(response_pattern.split(), match_result)

In [30]:
get_response('last night I was sleeping')

'Why do you tell me you were sleeping now?'

## 问题2： 考虑加入汉语回答情况

In [31]:
import jieba

第一步应该先完成将 汉语 和 特殊符号区分开来，下面的版本是傻瓜版本，代码不够优美，但是可是实现功能

In [32]:
def devide(string):
    result = []
    devi_chinese = ''
    devi_english = ''
    for ch in string:
        if u'\u4e00'<= ch <=u'\u9fff':
            devi_chinese += ch
            if devi_english != '':
                result += [devi_english]
            devi_english = ''
        else:
            devi_english += ch
            if devi_chinese != '':
                result += [devi_chinese]
            devi_chinese = ''
    if devi_chinese != '':
        result += [devi_chinese]
    else:
        result += [devi_english]
    return result

In [33]:
devide('这就行了?*x总是?*y来来**未来')

['这就行了', '?*x', '总是', '?*y', '来来', '**', '未来']

下一步，使用递归算法实现 devide 函数，代码变得优美许多，简洁很多，更加易于理解

In [34]:
def devide(string):
    for i in range(len(string)):
        if i == len(string)-1:
            return [] + [string]
        if bool(u'\u4e00'<=string[i]<=u'\u9fff') ^ bool(u'\u4e00'<=string[i+1]<=u'\u9fff'):
            return [string[:i+1]] + devide(string[i+1:])

In [35]:
devide('这就行了?*x总是?*y来来**未来')

['这就行了', '?*x', '总是', '?*y', '来来', '**', '未来']

### 思路如下：
1. 对于输入的句子，只能通过 jieba 分词来进行拆分，但是拆分结果是不是能够和预设的 pattern 匹配，是不是可能分词产生错误，这些问题都是不可控的，好在还有最后一个“装傻回答”，“['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']”
<br>

2. 对于 rule_responses 中的中英文混合情况，首先要将英语和汉语部分拆分，然后再对分出来的汉语词组进行 jieba 分词操作，这样产生问题的可能性会变小。

In [36]:
def cut_sentence(string):
    for ch in string:
        if u'\u4e00'<= ch <=u'\u9fff':
            return 0, list(jieba.cut(string))
    return 1, string.split()

In [37]:
def get_response(string, rules=rule_responses):
    state, string = cut_sentence(string)
    for k in rules:
        chi_k = []
        if state == 0:
            k_devide = devide(k)
            for k_devi in k_devide:
                if k_devi[0]<u'\u4e00' or k_devi[0] > u'\u9fff':
                    chi_k += [k_devi]
                else:
                    for chin in list(jieba.cut(k_devi)):
                        chi_k += [chin]
            match_result = pat_match_with_seg(chi_k, string)
        else:
            match_result = pat_match_with_seg(k.split(), string)
        if None not in match_result:
            response_pattern = random.choice(rules[k])
            response_pattern = devide(response_pattern)
            match_result = pat_to_dict(match_result)
            if state == 0:
                return substitude_chi(response_pattern, match_result)
            else:
                return substitude_eng(response_pattern.split(), match_result)

In [40]:
get_response('我一直都是傻傻呆呆的')

'真的---总是吗？'

In [41]:
get_response('昨天晚上我记得梦见一个特别特别可怕的人')

'你经常会想起这个吗？'

In [42]:
get_response('用机器人做这个')

'你为什么觉得机器人要解决你的问题？'

In [46]:
get_response('蚊子就像吸血鬼')

'你觉得蚊子和吸血鬼有什么相似性？'

In [47]:
get_response('哎呀不嘛！')

'你有一点负能量'

In [430]:
get_response('难道你是傻逼吗')

'那你希望我是傻逼吗'

In [48]:
get_response('感觉好难过呀')

'不应该让你这么难过的，你觉得你拥有什么，就会不难过?'

In [50]:
get_response('昨天我很难过，因为吵架了')

'吵架了不应该让你这么难过的'

## 问题3：多设计几个模式，让这个程序变得更好玩

设计一个“好奇宝宝”程序，可以无限次循环问下去

In [51]:
Curiosity_rules = {
    '?*x是?*p': ['为什么?x是?p？', '?x和?p有什么关系？', '什么是?x', '什么是?p', '真的假的？'],
    '?*x导致?*p': ['为什么?x导致?p？', '?x和?p关系么？', '什么是?x', '什么是?p', '真的假的？'],
    '?*x造成?*p': ['为什么?x造成?p？', '我觉得?x和?p没什么关系', '什么是?x', '什么是?p', '好假！'],
    '因为?*x所以?*p': ['为什么?x呢？','这有因果关系吗？','为什么从?x可以得到?p呢？'],
    '?*p因为?*x': ['由?x可以得到什么结论呢？', '感觉有点儿扯', '所以呢？', 'So?'],
    '?*x': ['为什么呢？', '这是怎么回事？', '好奇怪哦', '帮我解释解释为什么嘛...','好难懂，什么意思啊？']
}

In [52]:
get_response('好像出去玩啊', Curiosity_rules)

'这是怎么回事？'

In [53]:
get_response('因为天气好所以想出去玩啊', Curiosity_rules)

'为什么天气好呢？'

In [54]:
get_response('我是从天气预报知道的', Curiosity_rules)

'好奇怪哦'

In [57]:
get_response('天气预报是预测未来天气状况的啊', Curiosity_rules)

'真的假的？'

## 问题4： 
1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？

<bo>
    
2. 什么是数据驱动？数据驱动在这个程序里如何体现？

<bo>
    
3. 数据驱动与 AI 的关系是什么？

第二个问题：
数据驱动指的是，针对一类问题和数据，不需要更改程序，只要改变输入就能自发的改变输出。

这个程序的数据驱动性主要来源于每次变换语法规则，就能自发完成对话。

AI 解决的是一类问题，而这一类问题的随机出现一定不能通过简单地 if else 来判断，而是根据不同的数据，自发产生不同的